In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
!pwd

/home/theo/code/theopanel/bva/notebooks


In [3]:
video_url = "../bva/data/video_details.csv"
clean_df = "../bva/data/clean_dataset.csv"

In [4]:
df = pd.read_csv(clean_df)

In [5]:
df["player_A_visible"] = np.random.randint(100)
df["player_B_visible"] = np.random.randint(100)
df["player_A_court_x"] = np.random.randint(100)
df["player_A_court_y"] = np.random.randint(100)
df["player_B_court_x"] = np.random.randint(100)
df["player_B_court_y"] = np.random.randint(100)
df["player_A_img_x"] = np.random.randint(100)
df["player_A_img_y"] = np.random.randint(100)
df["player_B_img_x"] = np.random.randint(100)
df["player_B_img_y"] = np.random.randint(100)

In [6]:
video_details = pd.read_csv(video_url)

In [7]:
df = df.merge(video_details, on="video_path")

In [8]:
# fonction preprocess

def normalize_x(birdie_visible, birdie_x, width):
    if birdie_visible == 1:
        return birdie_x / width
    else:
        return -1

def normalize_y(birdie_visible, birdie_y, height):
    if birdie_visible == 1:
        return birdie_y / height
    else:
        return -1

def get_features(df_shots):
    court_width = 670
    court_height = 1340
    
    df_shots['birdie_x_nrm'] = df_shots.apply(
        lambda x: normalize_x(x.birdie_visible, x.birdie_x, x.width), axis=1)
    df_shots['birdie_y_nrm'] = df_shots.apply(
        lambda x: normalize_y(x.birdie_visible, x.birdie_y, x.height), axis=1)
    df_shots['ul_corner_x_nrm'] = df_shots.apply(
        lambda x: normalize_y(1, x.ul_corner_x, x.width), axis=1)
    df_shots['ul_corner_y_nrm'] = df_shots.apply(
        lambda x: normalize_y(1, x.ul_corner_y, x.height), axis=1)
    df_shots['ur_corner_x_nrm'] = df_shots.apply(
        lambda x: normalize_y(1, x.ur_corner_x, x.width), axis=1)
    df_shots['ur_corner_y_nrm'] = df_shots.apply(
        lambda x: normalize_y(1, x.ur_corner_y, x.height), axis=1)
    df_shots['br_corner_x_nrm'] = df_shots.apply(
        lambda x: normalize_y(1, x.br_corner_x, x.width), axis=1)
    df_shots['br_corner_y_nrm'] = df_shots.apply(
        lambda x: normalize_y(1, x.br_corner_y, x.height), axis=1)
    df_shots['bl_corner_x_nrm'] = df_shots.apply(
        lambda x: normalize_y(1, x.bl_corner_x, x.width), axis=1)
    df_shots['bl_corner_y_nrm'] = df_shots.apply(
        lambda x: normalize_y(1, x.bl_corner_y, x.height), axis=1)
    df_shots['player_A_court_x_nrm'] = df_shots.apply(
        lambda x: normalize_x(x.player_A_visible, x.player_A_court_x, court_width), axis=1)
    df_shots['player_A_court_y_nrm'] = df_shots.apply(
        lambda x: normalize_x(x.player_A_visible, x.player_A_court_y, court_height), axis=1)
    df_shots['player_B_court_x_nrm'] = df_shots.apply(
        lambda x: normalize_x(x.player_B_visible, x.player_B_court_x, court_width), axis=1)
    df_shots['player_B_court_y_nrm'] = df_shots.apply(
        lambda x: normalize_x(x.player_B_visible, x.player_B_court_y, court_height), axis=1)   
    df_shots['player_A_img_x_nrm'] = df_shots.apply(
        lambda x: normalize_x(x.player_A_visible, x.player_A_img_x, x.width), axis=1)
    df_shots['player_A_img_y_nrm'] = df_shots.apply(
        lambda x: normalize_y(x.player_A_visible, x.player_A_img_y, x.height), axis=1)
    df_shots['player_B_img_x_nrm'] = df_shots.apply(
        lambda x: normalize_x(x.player_B_visible, x.player_B_img_x, x.width), axis=1)
    df_shots['player_B_img_y_nrm'] = df_shots.apply(
        lambda x: normalize_y(x.player_B_visible, x.player_B_img_y, x.height), axis=1)
    return df_shots

In [9]:
df_shots = get_features(df)

In [10]:
videos = df_shots["video_path"].unique()

In [11]:
len(videos)

96

In [18]:
def get_video_sequences(video_frames):
    
    video_frames = video_frames.reset_index()
    
    sequences = []
    targets = []

    sequence_features = []
    idx = 0

    while idx < len(video_frames):
    #     print(idx)
        # no hit in 12 following frames
        if video_frames["birdie_hit"][idx:idx+12].sum() == 0:
#             print("no_hit")
            for ind, frame in video_frames[idx:idx+12].iterrows():
                features = [frame.birdie_visible,
                            frame.birdie_x_nrm,
                            frame.birdie_y_nrm,
                            frame.ul_corner_x_nrm,
                            frame.ul_corner_y_nrm,
                            frame.ur_corner_x_nrm,
                            frame.ur_corner_y_nrm,
                            frame.br_corner_x_nrm,
                            frame.br_corner_y_nrm,
                            frame.bl_corner_x_nrm,
                            frame.bl_corner_y_nrm,
                            frame.player_A_visible,
                            frame.player_B_visible,
                            frame.player_A_court_x_nrm,
                            frame.player_A_court_y_nrm,
                            frame.player_B_court_x_nrm,
                            frame.player_B_court_y_nrm,
                            frame.player_A_img_x_nrm,
                            frame.player_A_img_y_nrm,
                            frame.player_B_img_x_nrm,
                            frame.player_B_img_y_nrm]
                sequence_features.append(features)
            sequences.append(sequence_features)
            targets.append(0)
            # continue ?
            if len(video_frames[idx+12:])>= 12:
                sequence_features = []
                idx += 12
            else:
                break
        # hit detected in 12 frames
        else:
#             print("found hit")
            id_hit = video_frames[idx:idx+12].index[video_frames["birdie_hit"][idx:idx+12]==1].to_list()[0]
            if id_hit-6<0:
                idx +=1
                continue
            for ind, frame in video_frames[id_hit-6:id_hit+6].iterrows():
                features = [frame.birdie_visible,
                            frame.birdie_x_nrm,
                            frame.birdie_y_nrm,
                            frame.ul_corner_x_nrm,
                            frame.ul_corner_y_nrm,
                            frame.ur_corner_x_nrm,
                            frame.ur_corner_y_nrm,
                            frame.br_corner_x_nrm,
                            frame.br_corner_y_nrm,
                            frame.bl_corner_x_nrm,
                            frame.bl_corner_y_nrm,
                            frame.player_A_visible,
                            frame.player_B_visible,
                            frame.player_A_court_x_nrm,
                            frame.player_A_court_y_nrm,
                            frame.player_B_court_x_nrm,
                            frame.player_B_court_y_nrm,
                            frame.player_A_img_x_nrm,
                            frame.player_A_img_y_nrm,
                            frame.player_B_img_x_nrm,
                            frame.player_B_img_y_nrm]
                sequence_features.append(features)
            sequences.append(sequence_features)
            targets.append(video_frames["birdie_hit"][id_hit])
            # continue ?
            if len(video_frames[id_hit+6:])>= 12:
                sequence_features = []
                idx = id_hit+6
            else:
                break
    
    return np.array(sequences), np.array(targets)

In [19]:
#get sequences by video

all_videos_sequences = []
all_videos_targets = []

for video in videos:
    print(video)
    all_video_frames = df_shots[df_shots["video_path"]==video]
    X, y = get_video_sequences(all_video_frames)
    
    # add to results
    print(np.array(X).shape, np.array(y).shape)
    if len(all_videos_sequences) > 0:
        all_videos_sequences = np.vstack((all_videos_sequences, X))
        all_videos_targets = np.concatenate([all_videos_targets, y])
    else:
        all_videos_sequences = X
        all_videos_targets = y

match12/rally_video/1_01_00.mp4
(57, 12, 21) (57,)
match12/rally_video/1_10_12.mp4
(69, 12, 21) (69,)
match12/rally_video/2_05_14.mp4
(63, 12, 21) (63,)
match12/rally_video/3_03_03.mp4
(35, 12, 21) (35,)
match13/rally_video/1_09_10.mp4
(49, 12, 21) (49,)
match13/rally_video/1_17_15.mp4
(27, 12, 21) (27,)
match13/rally_video/2_06_05.mp4
(74, 12, 21) (74,)
match13/rally_video/2_07_05.mp4
(18, 12, 21) (18,)
match13/rally_video/2_09_08.mp4
(11, 12, 21) (11,)
match13/rally_video/2_17_11.mp4
(38, 12, 21) (38,)
match15/rally_video/1_21_12.mp4
(28, 12, 21) (28,)
match15/rally_video/2_14_08.mp4
(20, 12, 21) (20,)
match15/rally_video/2_16_12.mp4
(49, 12, 21) (49,)
match15/rally_video/2_18_14.mp4
(75, 12, 21) (75,)
match15/rally_video/2_19_14.mp4
(57, 12, 21) (57,)
match16/rally_video/1_03_06.mp4
(50, 12, 21) (50,)
match16/rally_video/1_13_20.mp4
(34, 12, 21) (34,)
match16/rally_video/2_08_08.mp4
(42, 12, 21) (42,)
match16/rally_video/3_14_09.mp4
(28, 12, 21) (28,)
match16/rally_video/3_17_16.mp4

In [38]:
unique, counts = np.unique(targets, return_counts=True)
unique, counts

(array([0, 1]), array([39, 25]))

In [ ]:
unique, counts = np.unique(targets, return_counts=True)
if counts[0] > counts[1]:
    #get X random indexes to delete from sequences and targets
    for nb_delete in range(counts[0]-counts[1]):
        print("x")

In [20]:
df_shots.columns

Index(['Unnamed: 0_x', 'video_path', 'frame', 'birdie_visible', 'birdie_x',
       'birdie_y', 'birdie_hit', 'stroke', 'player_A_visible',
       'player_B_visible', 'player_A_court_x', 'player_A_court_y',
       'player_B_court_x', 'player_B_court_y', 'player_A_img_x',
       'player_A_img_y', 'player_B_img_x', 'player_B_img_y', 'Unnamed: 0_y',
       'fps', 'width', 'height', 'ul_corner_x', 'ul_corner_y', 'ur_corner_x',
       'ur_corner_y', 'br_corner_x', 'br_corner_y', 'bl_corner_x',
       'bl_corner_y', 'left_net_x', 'left_net_y', 'right_net_x', 'right_net_y',
       'birdie_x_nrm', 'birdie_y_nrm', 'ul_corner_x_nrm', 'ul_corner_y_nrm',
       'ur_corner_x_nrm', 'ur_corner_y_nrm', 'br_corner_x_nrm',
       'br_corner_y_nrm', 'bl_corner_x_nrm', 'bl_corner_y_nrm',
       'player_A_court_x_nrm', 'player_A_court_y_nrm', 'player_B_court_x_nrm',
       'player_B_court_y_nrm', 'player_A_img_x_nrm', 'player_A_img_y_nrm',
       'player_B_img_x_nrm', 'player_B_img_y_nrm'],
      dtype='obj

In [33]:
df_shots[df_shots["video_path"]=="match13/rally_video/2_09_08.mp4"]

,Unnamed: 0_x,video_path,frame,birdie_visible,birdie_x,birdie_y,birdie_hit,stroke,player_A_visible,player_B_visible,...,bl_corner_x_nrm,bl_corner_y_nrm,player_A_court_x_nrm,player_A_court_y_nrm,player_B_court_x_nrm,player_B_court_y_nrm,player_A_img_x_nrm,player_A_img_y_nrm,player_B_img_x_nrm,player_B_img_y_nrm
4672,4672,match13/rally_video/2_09_08.mp4,0,0,0,0,0,NaN,41,19,...,0.13125,0.930556,-1,-1,-1,-1,-1,-1,-1,-1
4673,4673,match13/rally_video/2_09_08.mp4,1,0,0,0,0,NaN,41,19,...,0.13125,0.930556,-1,-1,-1,-1,-1,-1,-1,-1
4674,4674,match13/rally_video/2_09_08.mp4,2,0,0,0,0,NaN,41,19,...,0.13125,0.930556,-1,-1,-1,-1,-1,-1,-1,-1
4675,4675,match13/rally_video/2_09_08.mp4,3,1,673,300,0,NaN,41,19,...,0.13125,0.930556,-1,-1,-1,-1,-1,-1,-1,-1
4676,4676,match13/rally_video/2_09_08.mp4,4,1,673,296,1,short_serve,41,19,...,0.13125,0.930556,-1,-1,-1,-1,-1,-1,-1,-1
4677,4677,match13/rally_video/2_09_08.mp4,5,1,658,281,0,NaN,41,19,...,0.13125,0.930556,-1,-1,-1,-1,-1,-1,-1,-1
4678,4678,match13/rally_video/2_09_08.mp4,6,0,0,0,0,NaN,41,19,...,0.13125,0.930556,-1,-1,-1,-1,-1,-1,-1,-1
4679,4679,match13/rally_video/2_09_08.mp4,7,1,637,264,0,NaN,41,19,...,0.13125,0.930556,-1,-1,-1,-1,-1,-1,-1,-1
4680,4680,match13/rally_video/2_09_08.mp4,8,0,0,0,0,NaN,41,19,...,0.13125,0.930556,-1,-1,-1,-1,-1,-1,-1,-1
4681,4681,match13/rally_video/2_09_08.mp4,9,1,616,260,0,NaN,41,19,...,0.13125,0.930556,-1,-1,-1,-1,-1,-1,-1,-1
